In [32]:
import os
import sys
import pandas as pd
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from bayesian_dpddm.models import MLPModel, ConvModel, ResNetModel
from bayesian_dpddm.monitors import DPDDMBayesianMonitor
from experiments.utils import get_configs, get_datasets
from bayesian_dpddm.models import MLPModel, ConvModel
from data.uci_data import get_uci_datasets
from omegaconf import OmegaConf
from hydra import compose, initialize

# Point to your config directory and name
config_dir = "configs/"  # adjust as needed
config_name = "uci_best.yaml"

with initialize(config_path=config_dir, version_base='1.2'):
    args = compose(config_name=config_name)

# cfg is now a DictConfig
print(args)
dataset = get_datasets(args)
model_config, train_config = get_configs(args)
trainloader = torch.utils.data.DataLoader(
    dataset['train'],
    batch_size=train_config.batch_size,
    shuffle=True,
    num_workers=train_config.num_workers,
    #pin_memory=train_config.pin_memory,
    #persistent_workers=True,
)

valloader = torch.utils.data.DataLoader(
    dataset['valid'],
    batch_size=train_config.batch_size,
    shuffle=False,
    num_workers=train_config.num_workers,
    #pin_memory=train_config.pin_memory,
    #persistent_workers=True,
)

oodloader = torch.utils.data.DataLoader(
    dataset['dpddm_ood'],
    batch_size = train_config.batch_size,
    shuffle=False,
    num_workers=train_config.num_workers,
    #pin_memory=True, 
    #persistent_workers=True
)

{'train': {'disagreement_epochs': 5, 'disagreement_optimizer': 'torch.optim.AdamW', 'disagreement_wd': 0.0001, 'disagreement_lr': 0.01, 'disagreement_batch_size': 64, 'disagreement_alpha': 0.8, 'num_epochs': 50, 'batch_size': 64, 'lr': 0.001, 'wd': 0.0001, 'optimizer': 'torch.optim.AdamW', 'clip_val': 1, 'val_freq': 1, 'num_workers': 4, 'pin_memory': True}, 'dataset': {'name': 'uci', 'num_classes': 2, 'data_dir': 'data/uci_data', 'normalize': True}, 'dpddm': {'Phi_size': 1000, 'n_post_samples': 5000, 'data_sample_size': 10, 'temp': 1, 'n_repeats': 100}, 'model': {'name': 'mlp_model', 'in_features': 9, 'mid_features': 16, 'mid_layers': 4, 'dropout': 0.2, 'reg_weight_factor': 100, 'param': 'diagonal', 'prior_scale': 1.0, 'wishart_scale': 1.0, 'return_ood': False}, 'wandb_cfg': {'project': 'bayesian_dpddm', 'entity': 'viet', 'job_type': 'train', 'log_artifacts': True}, 'monitor_type': 'bayesian', 'from_pretrained': False, 'seed': 57, 'self_log': True}


# Accuracies on UCI

In [35]:
import os
import sys
#os.chdir('../')
os.getcwd()

'/mfs1/u/viet/bayesian_dpddm'

In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from bayesian_dpddm.models import MLPModel, ConvModel, ResNetModel


In [37]:
from experiments.utils import get_configs, get_datasets
from bayesian_dpddm.models import MLPModel, ConvModel
from data.uci_data import get_uci_datasets
from omegaconf import OmegaConf
from hydra import compose, initialize

# Point to your config directory and name
config_dir = "configs/"  # adjust as needed
config_name = "uci_best.yaml"

with initialize(config_path=config_dir, version_base='1.2'):
    args = compose(config_name=config_name)

# cfg is now a DictConfig
print(args)

{'train': {'disagreement_epochs': 5, 'disagreement_optimizer': 'torch.optim.AdamW', 'disagreement_wd': 0.0001, 'disagreement_lr': 0.01, 'disagreement_batch_size': 64, 'disagreement_alpha': 0.8, 'num_epochs': 50, 'batch_size': 64, 'lr': 0.001, 'wd': 0.0001, 'optimizer': 'torch.optim.AdamW', 'clip_val': 1, 'val_freq': 1, 'num_workers': 4, 'pin_memory': True}, 'dataset': {'name': 'uci', 'num_classes': 2, 'data_dir': 'data/uci_data', 'normalize': True}, 'dpddm': {'Phi_size': 1000, 'n_post_samples': 5000, 'data_sample_size': 10, 'temp': 1, 'n_repeats': 100}, 'model': {'name': 'mlp_model', 'in_features': 9, 'mid_features': 16, 'mid_layers': 4, 'dropout': 0.2, 'reg_weight_factor': 100, 'param': 'diagonal', 'prior_scale': 1.0, 'wishart_scale': 1.0, 'return_ood': False}, 'wandb_cfg': {'project': 'bayesian_dpddm', 'entity': 'viet', 'job_type': 'train', 'log_artifacts': True}, 'monitor_type': 'bayesian', 'from_pretrained': False, 'seed': 57, 'self_log': True}


In [38]:
dataset = get_datasets(args)
model_config, train_config = get_configs(args)
trainloader = torch.utils.data.DataLoader(
    dataset['train'],
    batch_size=train_config.batch_size,
    shuffle=True,
    num_workers=train_config.num_workers,
    #pin_memory=train_config.pin_memory,
    #persistent_workers=True,
)

valloader = torch.utils.data.DataLoader(
    dataset['valid'],
    batch_size=train_config.batch_size,
    shuffle=False,
    num_workers=train_config.num_workers,
    #pin_memory=train_config.pin_memory,
    #persistent_workers=True,
)

oodloader = torch.utils.data.DataLoader(
    dataset['dpddm_ood'],
    batch_size = train_config.batch_size,
    shuffle=False,
    num_workers=train_config.num_workers,
    #pin_memory=True, 
    #persistent_workers=True
)

In [39]:
model = MLPModel(model_config, train_size=len(dataset['train'])).cuda()
model.out_layer = nn.Linear(model_config.mid_features, model_config.out_features).cuda()
opt = torch.optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

In [48]:
for epoch in tqdm(range(train_config.num_epochs)):
    model.train()
    for batch in tqdm(trainloader, leave=False):
        features, labels, *_ = batch
        features, labels = features.cuda(), labels.cuda()
        out = model(features)
        loss = criterion(out, labels.long())
        opt.zero_grad()
        loss.backward()
        opt.step()

100%|██████████| 50/50 [00:05<00:00,  8.75it/s]


In [45]:
def evaluate(model, loader):
    model.eval()
    predictions = []
    ll = []
    with torch.no_grad():
        for batch in loader:
            features, labels, *_ = batch
            features, labels = features.cuda(), labels.cuda()
            out = model(features)
            preds = torch.argmax(out, dim=-1)
            predictions.append(preds.cpu())
            ll.append(labels.cpu())
    predictions = torch.cat(predictions).numpy().squeeze()
    ll = torch.cat(ll).numpy()
    return (predictions == ll).mean()
            

In [46]:
id_perf, ood_perf = [], []
for i in range(10):
    print('round {}'.format(i))
    model = MLPModel(model_config, train_size=len(dataset['train'])).cuda()
    model.out_layer = nn.Linear(model_config.mid_features, model_config.out_features).cuda()
    opt = torch.optim.AdamW(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()
    for epoch in tqdm(range(50), leave=False):
        model.train()
        for batch in trainloader:
            features, labels, *_ = batch
            features, labels = features.cuda(), labels.cuda()
            out = model(features)
            loss = criterion(out, labels.long())
            opt.zero_grad()
            loss.backward()
            opt.step()
    acc_id = evaluate(model, valloader)
    acc_ood = evaluate(model, oodloader)
    id_perf.append(acc_id)
    ood_perf.append(acc_ood)

round 0


round 1


round 2


round 3


round 4


round 5


round 6


round 7


round 8


round 9


In [47]:
import numpy as np
np.mean(id_perf), np.mean(ood_perf), np.mean(id_perf) - np.mean(ood_perf)

(np.float64(0.7733333333333334),
 np.float64(0.6609907120743033),
 np.float64(0.11234262125903016))

In [49]:
np.std(np.array(id_perf) - np.array(ood_perf))

np.float64(0.015065534047155943)

# VBLL vs Regular Head Classifier

In [19]:
df = pd.read_csv('ablations/ablation_results/reghead/reghead.csv')
df = df[df['dataset'] == 'camelyon17']
df[df['model'] == 'D3M'].describe(), df[df['model'] == 'Reg'].describe()

(         run_id   accuracy   f1_score      auroc        mcc
 count  10.00000  10.000000  10.000000  10.000000  10.000000
 mean    4.50000   0.940323   0.940246   0.984192   0.881923
 std     3.02765   0.008584   0.008745   0.000734   0.014476
 min     0.00000   0.916179   0.915637   0.982470   0.841401
 25%     2.25000   0.941805   0.941752   0.984369   0.884487
 50%     4.50000   0.943193   0.943176   0.984464   0.886653
 75%     6.75000   0.943764   0.943732   0.984586   0.888114
 max     9.00000   0.944549   0.944532   0.984712   0.889296,
          run_id   accuracy   f1_score      auroc        mcc
 count  10.00000  10.000000  10.000000  10.000000  10.000000
 mean    4.50000   0.941696   0.941674   0.983425   0.883673
 std     3.02765   0.000498   0.000494   0.000167   0.001102
 min     0.00000   0.940813   0.940801   0.983159   0.881716
 25%     2.25000   0.941391   0.941367   0.983313   0.882986
 50%     4.50000   0.941772   0.941752   0.983406   0.883771
 75%     6.75000   0.94

## UCI Heart Disease

In [ ]:
df = pd.read_csv('ablation_results/reghead/reghead.csv')
df = df[df['dataset'] == 'uci']
df[df['model'] == 'D3M'].describe()

In [ ]:
df[df['model'] == 'D3M'].describe()

,run_id,accuracy,f1_score,auroc,mcc
count,10.00000,10.000000,10.000000,10.000000,10.000000
mean,4.50000,0.761667,0.751474,0.855458,0.511780
std,3.02765,0.019720,0.021929,0.010792,0.041304
min,0.00000,0.725000,0.712523,0.844174,0.433709
25%,2.25000,0.754167,0.738308,0.844881,0.497221
50%,4.50000,0.766667,0.756059,0.853507,0.521132
75%,6.75000,0.766667,0.758551,0.865314,0.522866
max,9.00000,0.800000,0.793044,0.871041,0.590309


: 

In [ ]:
df[df['model'] == 'Reg'].describe()

,run_id,accuracy,f1_score,auroc,mcc
count,10.00000,10.000000,10.000000,10.000000,10.000000
mean,4.50000,0.767500,0.757954,0.867477,0.523324
std,3.02765,0.011419,0.011956,0.006427,0.024104
min,0.00000,0.750000,0.739583,0.856900,0.486236
25%,2.25000,0.760417,0.750146,0.862557,0.508044
50%,4.50000,0.766667,0.756862,0.867930,0.521472
75%,6.75000,0.772917,0.763231,0.871324,0.534995
max,9.00000,0.791667,0.782214,0.876980,0.573992


: 

## CIFAR-10

In [ ]:
df = pd.read_csv('ablation_results/reghead/reghead.csv')
df = df[df['dataset'] == 'cifar10']

: 

In [ ]:
df[df['model'] == 'D3M'].describe()

,run_id,accuracy,f1_score,auroc,mcc
count,10.00000,10.000000,10.000000,10.000000,10.000000
mean,4.50000,0.700320,0.697698,0.955781,0.668806
std,3.02765,0.020199,0.020833,0.004538,0.021162
min,0.00000,0.647300,0.643624,0.944173,0.613371
25%,2.25000,0.697675,0.693627,0.955177,0.665914
50%,4.50000,0.703700,0.700444,0.956905,0.672835
75%,6.75000,0.711550,0.710807,0.958159,0.680743
max,9.00000,0.717800,0.714789,0.959723,0.687447


: 

In [ ]:
df[df['model'] == 'Reg'].describe()

,run_id,accuracy,f1_score,auroc,mcc
count,10.00000,10.000000,10.000000,10.000000,10.000000
mean,4.50000,0.719140,0.717035,0.960638,0.689076
std,3.02765,0.012803,0.012988,0.001908,0.013839
min,0.00000,0.701800,0.697273,0.957714,0.670288
25%,2.25000,0.707100,0.706406,0.959589,0.675893
50%,4.50000,0.721150,0.718639,0.960844,0.691617
75%,6.75000,0.727500,0.725275,0.961269,0.698249
max,9.00000,0.738500,0.737607,0.963903,0.710044


: 

In [26]:
df = pd.read_csv('ablations/ablation_results/temp/temp.csv')

In [27]:
uci = df[df['dataset'] == 'uci']
cam = df[df['dataset'] == 'camelyon17']
cif = df[df['dataset'] == 'cifar10']

In [28]:
uci, cif, cam = list(map(lambda x : x.drop(columns=['seed', 'data_sample_size', 'fpr_train', 'dis_rate_train_std', 'dis_rate_id_std', 'dataset', 'dis_rate_train_mean','dis_rate_ood_std']),  [uci, cif, cam]))

In [29]:
uci

,temp,fpr_id,tpr,dis_rate_id_mean,dis_rate_ood_mean
0,1,0.01,0.85,0.4359,0.4919
1,2,0.07,0.49,0.5478,0.5785
2,3,0.07,0.32,0.5864,0.6089
3,5,0.09,0.12,0.6280,0.6381
4,10,0.08,0.19,0.6520,0.6615
5,20,0.04,0.11,0.6679,0.6721


In [30]:
cif

,temp,fpr_id,tpr,dis_rate_id_mean,dis_rate_ood_mean
6,1,0.14,0.98,0.4309,0.5189
7,2,0.06,1.00,0.6405,0.7267
8,3,0.06,0.96,0.7683,0.8289
9,5,0.05,0.78,0.8837,0.9136
10,10,0.05,0.34,0.9505,0.9619
11,20,0.07,0.09,0.9744,0.9762


In [31]:
cam

,temp,fpr_id,tpr,dis_rate_id_mean,dis_rate_ood_mean
12,1,0.09,0.99,0.1696,0.2561
13,2,0.12,1.00,0.2891,0.3904
14,3,0.09,1.00,0.3801,0.4712
15,5,0.07,1.00,0.4774,0.5471
16,10,0.08,0.61,0.5803,0.6118
17,20,0.05,0.32,0.6295,0.6483
